In [1]:
import requests
import pandas as pd

In [2]:
url = "https://data.cityofchicago.org/resource/crimes.json"
response = requests.get(url)

In [3]:
if response.status_code == 200:
    datos = response.json()
    print(datos)
else:
    print("descargar archivo")

[{'id': '13000913', 'case_number': 'JG174323', 'date': '2023-03-04T23:57:00.000', 'block': '059XX S FRANCISCO AVE', 'iucr': '0560', 'primary_type': 'ASSAULT', 'description': 'SIMPLE', 'location_description': 'RESIDENCE', 'arrest': True, 'domestic': True, 'beat': '0824', 'district': '008', 'ward': '16', 'community_area': '66', 'fbi_code': '08A', 'x_coordinate': '1158062', 'y_coordinate': '1865140', 'year': '2023', 'updated_on': '2023-03-11T15:48:26.000', 'latitude': '41.785689405', 'longitude': '-87.696013884', 'location': {'latitude': '41.785689405', 'longitude': '-87.696013884', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}, ':@computed_region_awaf_s7ux': '19', ':@computed_region_6mkv_f3dw': '21867', ':@computed_region_vrxf_vc4k': '64', ':@computed_region_bdys_3d7i': '183', ':@computed_region_43wa_7qmu': '2', ':@computed_region_rpca_8um6': '58', ':@computed_region_d9mm_jgwp': '13', ':@computed_region_d3ds_rm58': '269'}, {'id': '13000902', 'case_number': 'JG17

In [4]:
chicago = pd.read_json('https://data.cityofchicago.org/resource/crimes.json')

In [5]:
crimes_by_district = chicago.groupby(['district', 'description']).size().reset_index(name='count')

In [6]:
district_danger_scores = crimes_by_district.groupby('district').agg({'count': 'sum'})
district_danger_scores['danger_index'] = district_danger_scores['count'] / district_danger_scores['count'].max()

In [7]:
import folium

chicago_map = folium.Map(location=[41.8781, -87.6298], zoom_start=10)

In [8]:
for district in chicago['district'].unique():
    danger_score = district_danger_scores.loc[district]['danger_index']
    color = 'green' if danger_score < 0.7 else 'red'
    folium.CircleMarker(location=[chicago.loc[chicago['district'] == district]['latitude'].mean(),
                                   chicago.loc[chicago['district'] == district]['longitude'].mean()],
                        radius=10,
                        color=color,
                        fill=True,
                        fill_color=color).add_to(chicago_map)

chicago_map

In [9]:
import pandas as pd
import folium

In [10]:
chicago.dropna(subset=['latitude', 'longitude'], inplace=True)

In [11]:
chicago_map = folium.Map(location=[41.8781, -87.6298], zoom_start=60)

In [12]:
crimes = chicago['primary_type'].unique()
crime_counts = chicago.groupby(['primary_type']).size().reset_index(name='counts')

In [13]:
crime_dict = {}
for index, row in crime_counts.iterrows():
    crime_dict[row['primary_type']] = row['counts']

In [14]:
def get_color(primary_type):
    if crime_dict[primary_type] == max(crime_dict.values()):
        return 'red'
    elif crime_dict[primary_type] == min(crime_dict.values()):
        return 'grey'
    else:
        return 'green'

In [15]:
for index, row in chicago.iterrows():
    folium.PolyLine(
        locations=[(row['latitude'], row['longitude']), (row['latitude']+0.0002, row['longitude']+0.0002)],
        color=get_color(row['primary_type']),
        weight=5
    ).add_to(chicago_map)
    
chicago_map

In [17]:
menos_graves = chicago[chicago["description"].isin(["THEFT", "BATTERY", "CRIMINAL DAMAGE", "ASSAULT", "DECEPTIVE PRACTICE"])]
mas_graves = chicago[chicago["description"].isin(["HANDGUN", "CRIMINAL SEXUAL ASSAULT", "ROBBERY", "HOMICIDE", "MOTOR VEHICLE THEFT", "WEAPONS VIOLATION"])]

In [18]:
chicago_map = folium.Map(location=[chicago["latitude"].mean(), chicago["longitude"].mean()], zoom_start=60)

In [19]:
from folium.plugins import MarkerCluster

mc = MarkerCluster()

for idx, row in chicago.iterrows():
    mc.add_child(folium.Marker(location=[row["latitude"], row["longitude"]], popup=row["description"]))

chicago_map.add_child(mc)

In [20]:
from folium import FeatureGroup, LayerControl

fg_menos_graves = FeatureGroup(name="Delitos menos graves")
fg_mas_graves = FeatureGroup(name="Delitos más graves")

for idx, row in menos_graves.iterrows():
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], color="green", radius=5).add_to(fg_menos_graves)

for idx, row in mas_graves.iterrows():
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], color="red", radius=5).add_to(fg_mas_graves)

chicago_map.add_child(fg_menos_graves)
chicago_map.add_child(fg_mas_graves)
chicago_map.add_child(LayerControl())